In [15]:
include("../LiPoSID.jl")

#include("CtrlSID.jl")


using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra

using HDF5
using DynamicPolynomials

using Dates

using Statistics

using TSSOS

In [16]:
function frobenius_norm2(m)
    return tr(m * m')
end

frobenius_norm2 (generic function with 1 method)

In [17]:
function kossak_obj(ρ, t, Hˢʸᵐᵇ, Cˢʸᵐᵇ, Fᴼᴺᴮ)

    function Dc(ρ, t)
        U = (Hˢʸᵐᵇ*ρ - ρ*Hˢʸᵐᵇ)/im 
        D = sum(Cˢʸᵐᵇ .* [2*fᵢ*ρ*fⱼ' - ρ*fⱼ'*fᵢ - fⱼ'*fᵢ*ρ  for fᵢ in Fᴼᴺᴮ, fⱼ in Fᴼᴺᴮ])/2
        return U + D
    end 

    obj = 0
    for i in 3:length(ρ)
        obj += LiPoSID.frobenius_norm2(
            ρ[i] - ρ[i-2] - (t[i]-t[i-1])*(Dc(ρ[i], t[i])+
            4*Dc(ρ[i-1], t[i-1])+Dc(ρ[i-2], t[i-2]))/3
        )
    end

    if isempty(monomials(obj))
        obj = 0. 
    else
        obj = sum(real(coef) * mon for (coef, mon) in zip(coefficients(obj), monomials(obj)))
    end

    return obj

end

kossak_obj (generic function with 1 method)

In [18]:
function kossak_GEXY_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)

    ρᵍ, ρᵉ, ρˣ, ρʸ = ρᵍᵉˣʸ
    tᵍ, tᵉ, tˣ, tʸ = tᵍᵉˣʸ

    polyG = kossak_obj(ρᵍ, tᵍ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyE = kossak_obj(ρᵉ, tᵉ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyX = kossak_obj(ρˣ, tˣ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyY = kossak_obj(ρʸ, tʸ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)

    polyGEXY = polyG + polyE + polyX + polyY

    return polyGEXY
end

kossak_GEXY_obj (generic function with 1 method)

### Kossakowski objective for two-level system


For the two level system we can use theorem originally proved by Kossakowski:

Kossakowski, Bull. Acad. Pol. Sci. Ser. Math. Astr. Phys. 21, 649 (1973).

and available also in classic GKS paper:

Completely positive dynamical semigroups of Nlevel systems
Vittorio Gorini, Andrzej Kossakowski, and E. C. G. Sudarshan
Citation: J. Math. Phys. 17, 821 (1976); doi: 10.1063/1.522979

Theorem 3.1. (two-level system)

A linear operator $L: M(2) \rightarrow M(2)$ is the generator of a dynamical semigroup $t \rightarrow \Lambda_t$ of $M(2)$ iff it can be written in the form:

$L = -i[H,\rho] + \frac{1}{2} \sum_{i,j}^3 C_{ij} \{[f_i, \rho f_j] + [f_i\rho, f_j]\}, \quad \rho \in M(2)$

where

(i) $H = \sum_1^3 h_i f_i, \quad h_i \in \R$

(ii) $f_i f_j = \frac{1}{4}\delta_{ij}I  +\frac{i}{2} \sum_{k=1}^3 \epsilon_{ijk} f_k$ which means:

$ \operatorname{tr}(f_i f_j)=\frac{1}{2}\delta_{ij}, \quad \operatorname{tr}(f_i) = 0$


(iii) $\{C_{ij}\}= \begin{pmatrix} 
\gamma - 2\gamma_1    & - i a_3            &  i a_2 \\
i a_3                 & \gamma - 2\gamma_2 & -i a_1 \\
-i a_2                & i a_1              & \gamma - 2\gamma_3 
\end{pmatrix} =$

$=\begin{pmatrix} 
-\gamma_1 + \gamma_2 + \gamma_3   & - i a_3   &  i a_2 \\
i a_3                 & \gamma_1 - \gamma_2 + \gamma_3 & -i a_1 \\
-i a_2                & i a_1      & \gamma_1 + \gamma_2 - \gamma_3 
\end{pmatrix}$

To write a Hamiltonian $H$ and than objective function we can use the set of orthonormal operators $f_j$

We use Pauli matrices $\sigma_x, \sigma_y, \sigma_z$ to choose 3 orthonormal $f_i$ for the two-level system to fulfill:

$f_i f_j = \frac{1}{4}\delta_{ij}I  +\frac{i}{2} \sum_{k=1}^3 \epsilon_{ijk} f_k$ which means:

$ \operatorname{tr}(f_i f_j)=\frac{1}{2}\delta_{ij}, \quad \operatorname{tr}(f_i) = 0$

In [19]:
σˣ = [ 0 1 
       1 0 ]

σʸ = [ 0.   im*1
      -im*1 0    ]

σᶻ = [ 1.  0
       0  -1 ] 

fᴷ₁ = σˣ/2
fᴷ₂ = σʸ/2
fᴷ₃ = σᶻ/2

@assert tr(σˣ/2*σʸ/2) == tr(σˣ/2*σᶻ/2) ==  tr(σʸ/2*σᶻ/2) ≈ 0
@assert tr(σˣ/2*σˣ/2) == tr(σʸ/2*σʸ/2) == tr(σᶻ/2*σᶻ/2) ≈ 1/2

fᴷᴼᴺᴮ = [fᴷ₁, fᴷ₂, fᴷ₃]

3-element Vector{Matrix{ComplexF64}}:
 [0.0 + 0.0im 0.5 + 0.0im; 0.5 + 0.0im 0.0 + 0.0im]
 [0.0 + 0.0im 0.0 + 0.5im; 0.0 - 0.5im 0.0 + 0.0im]
 [0.5 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im -0.5 + 0.0im]

In [20]:
@polyvar ϵ h_Re h_Im # h₁ h₂ h₃

H0ˢʸᵐᵇ = [ ϵ               h_Re+im*h_Im
           h_Re-im*h_Im   -ϵ            ] / 2

H0ᴷˢʸᵐᵇ = h_Re * fᴷ₁ + h_Im * fᴷ₂  + ϵ * fᴷ₃ 

@assert tr(H0ᴷˢʸᵐᵇ) == 0
@assert H0ᴷˢʸᵐᵇ == H0ˢʸᵐᵇ

Hˢʸᵐᵇ = H0ˢʸᵐᵇ 

#@polyvar d_Re d_Im

#Vˢʸᵐᵇ =  [ 0             d_Re+im*d_Im
#           d_Re-im*d_Im  0.           ]    

@polyvar γ[1:3]
@polyvar a[1:3]

Cˢʸᵐᵇ = [ -γ[1]+γ[2]+γ[3]   -im*a[3]          im*a[2]
           im*a[3]           γ[1]-γ[2]+γ[3]  -im*a[1] 
          -im*a[2]           im*a[1]          γ[1]+γ[2]-γ[3]] 

κ₁ = -γ[1]+γ[2]+γ[3]
κ₂ =  γ[1]-γ[2]+γ[3]
κ₃ =  γ[1]+γ[2]-γ[3]

constr1 = κ₁ + κ₂ + κ₃  
constr2 = κ₁*κ₂ + κ₃*κ₁ + κ₂*κ₃ - - a[1]^2 - a[2]^2 - a[3]^2
constr3 = κ₁*κ₂*κ₃ - κ₁*a[1]^2 - κ₂*a[2]^2 - κ₃*a[3]^2;

constraints = [ϵ, γ[1], γ[2], γ[3], κ₁, κ₂, κ₃, constr1, constr2, constr3]

10-element Vector{Polynomial{true, Int64}}:
 ϵ
 γ₁
 γ₂
 γ₃
 -γ₁ + γ₂ + γ₃
 γ₁ - γ₂ + γ₃
 γ₁ + γ₂ - γ₃
 γ₁ + γ₂ + γ₃
 -γ₁² + 2γ₁γ₂ + 2γ₁γ₃ - γ₂² + 2γ₂γ₃ - γ₃² + a₁² - a₂² - a₃²
 -γ₁³ + γ₁²γ₂ + γ₁²γ₃ + γ₁γ₂² - 2γ₁γ₂γ₃ + γ₁γ₃² + γ₁a₁² - γ₁a₂² - γ₁a₃² - γ₂³ + γ₂²γ₃ + γ₂γ₃² - γ₂a₁² + γ₂a₂² - γ₂a₃² - γ₃³ - γ₃a₁² - γ₃a₂² + γ₃a₃²

In [21]:

function read_timeevolution(file_name, state, γ)
    h5open(file_name, "r") do file
        ρᵧ = read(file[state][string(γ)])
        t = ρᵧ["t"]
        ρ₀₀ = ρᵧ["p0"]; Re_ρ₀₁ = ρᵧ["s_re"];  Im_ρ₀₁ = ρᵧ["s_im"]
        ρ_series = []
        t_series = []

        for i in 1:length(t)
            ρᵢ= [ ρ₀₀[i]                      Re_ρ₀₁[i] + im * Im_ρ₀₁[i]
                  Re_ρ₀₁[i] - im * Im_ρ₀₁[i]  1 - ρ₀₀[i]                 ]
            push!(ρ_series, convert(Matrix{ComplexF64}, ρᵢ))
            push!(t_series, convert(Float64, t[i]))
        end
        return(t_series, ρ_series)
    end
end


function read_GEXY_timeevolution(file_name, γ)

    tᵍ, ρᵍ = read_timeevolution(file_name, "B1", γ)
    tᵉ, ρᵉ = read_timeevolution(file_name, "B2", γ)
    tˣ, ρˣ = read_timeevolution(file_name, "B3", γ)
    tʸ, ρʸ = read_timeevolution(file_name, "B4", γ)

    ρᵍᵉˣʸ = ρᵍ, ρᵉ, ρˣ, ρʸ 
    tᵍᵉˣʸ = tᵍ, tᵉ, tˣ, tʸ

    return tᵍᵉˣʸ , ρᵍᵉˣʸ 

end

read_GEXY_timeevolution (generic function with 1 method)

In [22]:
function TrDist(ρ₁, ρ₂)
    A = ρ₁-ρ₂
    D = tr(sqrt(A'*A))/2
    if abs(imag(D))>1e-6
        throw(DomainError(D, "Trace distance is complex number"))
    else
        return(real(D))
    end
end 

TrDist (generic function with 1 method)

In [23]:
function get_lindblad_operators(C, basis_ops)#(C::Matrix{ComplexF64}, basis_ops::Vector{Matrix{ComplexF64}})
    # Check that C is a square matrix and basis_ops has the same dimension
    n = size(C, 1)
    if size(C, 2) != n || length(basis_ops) != n
        throw(ArgumentError("Dimensions of C and basis_ops do not match"))
    end

    # Perform eigenvalue decomposition of C
    eigvals, eigvecs = eigen(C)

    # Construct the Lindblad operators
    lindblad_ops = []
    for i in 1:n
        if eigvals[i] > 1e-10  # Filter out negligible eigenvalues to ensure numerical stability
            lindblad_op = zeros(ComplexF64, size(basis_ops[1]))
            for j in 1:n
                lindblad_op .+= sqrt(eigvals[i]) * eigvecs[j, i] * basis_ops[j]
            end
            push!(lindblad_ops, lindblad_op)
        end
    end

    return lindblad_ops
end

get_lindblad_operators (generic function with 1 method)

In [24]:
#tests_dir = "../TESTS/"
tests_dir = ""

ρᵍ₀ = [ 1 0
        0 0 ]    # state to measure initial distance from

dodeca_10_states = ["D"*string(n) for n=1:10];

basis_states = ["B"*string(n) for n=1:4];

train_states = basis_states 
test_states = dodeca_10_states

all_states = vcat(train_states, test_states);

In [25]:
function filter_terms_by_relative_threshold(poly::Polynomial, relative_threshold::Float64)
    # Get all coefficients of the polynomial
    coeffs = coefficients(poly)
    
    # Find the largest coefficient by absolute value
    max_coeff = maximum(abs.(coeffs))
    
    # Calculate the effective threshold
    threshold = relative_threshold * max_coeff
    
    # Initialize an empty polynomial of the same type as the input
    new_poly = zero(poly)
    
    # Iterate over the terms and coefficients of the polynomial
    for (monomial, coeff) in zip(monomials(poly), coeffs)
        if abs(coeff) >= threshold
            new_poly += coeff * monomial
        end
    end
    
    return new_poly
end

# Example usage
#@polyvar x y
#p = 1e-13*x^2 + 2*x*y + 3*y^2 + 4e-14*y
#relative_threshold = 1e-12
#filtered_p = filter_terms_by_relative_threshold(p, relative_threshold)
#println(filtered_p)

filter_terms_by_relative_threshold (generic function with 1 method)

In [26]:

function filter_odd_terms_by_relative_threshold(poly::Polynomial, relative_threshold::Float64)
    # Get all coefficients and corresponding monomials of the polynomial
    coeffs = coefficients(poly)
    monoms = monomials(poly)
    
    # Find the largest coefficient by absolute value
    max_coeff = maximum(abs.(coeffs))
    
    # Calculate the effective threshold
    threshold = relative_threshold * max_coeff
    
    # Initialize an empty polynomial of the same type as the input
    new_poly = zero(poly)
    
    # Iterate over the terms and coefficients of the polynomial
    for (monomial, coeff) in zip(monoms, coeffs)
        # Check if the coefficient is above the threshold
        if abs(coeff) >= threshold
            # Add the term to the new polynomial
            new_poly += coeff * monomial
        else
            # Check if the monomial has only odd powers
            powers = exponents(monomial)
            if all(p -> p % 2 == 0, powers)
                # Add the term if it has even powers only
                new_poly += coeff * monomial
            end
        end
    end
    
    return new_poly
end

filter_odd_terms_by_relative_threshold (generic function with 1 method)

In [27]:
# Example usage
#@polyvar x y
#p = 1e-13*x^2 + 2*x*y + 3*y^2 + 4e-14*y + 4e-13*y^3
#relative_threshold = 1e-12
#filtered_p = filter_odd_terms_by_relative_threshold(p, relative_threshold)
#println(filtered_p)

1.0e-13*x^2 + 2.0*x*y + 3.0*y^2


In [13]:
function coefficient_range(poly::Polynomial)
    # Extract coefficients of the polynomial
    coeffs = coefficients(poly)
    
    # Get the absolute values of the coefficients
    abs_coeffs = abs.(coeffs)
    
    # Find the minimum and maximum coefficients in terms of magnitude
    min_coeff = minimum(abs_coeffs)
    max_coeff = maximum(abs_coeffs)
    
    # Calculate the ratio of the smallest to the largest coefficient
    if max_coeff == 0
        return 0.0  # Avoid division by zero if all coefficients are zero
    else
        return min_coeff / max_coeff
    end
end

coefficient_range (generic function with 1 method)

In [14]:
evol_data_file_name = "../DATA/ALL_GAMMAS_B4_D10.h5"

γᵢ = "0.25133"

tᵍᵉˣʸ , ρᵍᵉˣʸ  = read_GEXY_timeevolution(evol_data_file_name, γᵢ)
polyGEXYfull = kossak_GEXY_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)

0.10341163322607885ϵ² - 4.104630886766538e-5ϵh_Re + 3.451755329501592e-5ϵh_Im + 0.0005179676236185148ϵγ₁ - 0.0005179676236185148ϵγ₂ + 0.0012450858989678402ϵa₁ - 0.0008044950814777315ϵa₂ + 1.0519096548020683h_Re² + 0.0005179676236185148h_Reh_Im + 3.451755329501592e-5h_Reγ₂ - 3.45175532950157e-5h_Reγ₃ - 2.1131584754297306h_Rea₂ - 0.0012450858989678404h_Rea₃ + 1.0518715157602614h_Im² + 4.104630886766538e-5h_Imγ₁ - 4.104630886766538e-5h_Imγ₃ + 2.1131584754297306h_Ima₁ + 0.0008044950814777305h_Ima₃ + 0.05168674709213622γ₁² - 0.0008044950814777315γ₁a₁ + 0.05172488613394287γ₂² - 0.0012450858989678402γ₂a₂ + 1.0001847686681244γ₃² + 2.1131584754297306γ₃a₃ + 1.269688888901564a₁² - 8.005395128533376e-37a₁a₂ - 4.658513032068184e-37a₁a₃ + 1.269688888901564a₂² + 4.647304221135261e-36a₂a₃ + 1.269688888901564a₃² - 5.199679015672933ϵ + 0.00013669789552729324h_Re - 0.00035392288736811996h_Im - 0.013017661776398296γ₁ - 0.012979980165655292γ₂ + 0.02681159496728833γ₃ - 0.0312922847698259a₁ + 0.0210157103323

In [15]:
coefficient_range(polyGEXYfull)

7.125837652874591e-39

In [16]:
function scaling_poly(p::Polynomial)
    X = transpose(hcat([exponents(t) for t in terms(p)]...))

    # Get the scaling via linear regression
    scaling = X \ log.(abs.(coefficients(p)))

    exp.(abs.(scaling))
end

scaling_poly (generic function with 1 method)

In [17]:
function min_cs_tssos_scaled(p, constrs)

    coeffs = coefficients(p)

    # find variable scaling
    scale = scaling_poly(p)

    # scale the polynomial
    p_scaled = subs(p, variables(p) => scale .* variables(p))
    constrs_scaled = subs.(constrs, variables(p) => scale .* variables(p))

    # minimize
    reg_coef = 0.0
    # minimizer_scaled_tssos = nothing
    pop =[p_scaled+reg_coef*sum(variables(p).^2), constrs_scaled...] ./ maximum(abs.(coeffs))

    d = maxdegree(p)
    
    # Initial optimization step
    opt, sol, data = cs_tssos_first(pop, variables(pop), d; solution=true, QUIET=true)
    ref_sol, flag = TSSOS.refine_sol(opt, sol, data; QUIET=true)
    prev_opt, prev_sol, prev_data = opt, sol, data 

    # Check if the solution needs further refinement
    if flag != 0
        while ~isnothing(sol) && flag != 0
            prev_opt, prev_sol, prev_data = opt, sol, data
            opt, sol, data = cs_tssos_higher!(data; solution=true, QUIET=true) 
        end
        ref_sol, flag = TSSOS.refine_sol(prev_opt, prev_sol, prev_data; QUIET=true)
    end

    minimizer_scaled_tssos = scale .* ref_sol

    solution = variables(p) => minimizer_scaled_tssos

    if flag == 0 
        status_name = "GLOBAL"
    else
        status_name = "LOCAL/FAIL"
    end

    return solution, status_name

end
  

min_cs_tssos_scaled (generic function with 1 method)

In [18]:
solution, status_name = min_cs_tssos_scaled(polyGEXYfull, constraints)

*********************************** TSSOS ***********************************
TSSOS is launching...
-----------------------------------------------------------------------------
The clique sizes of varibles:
[8, 7]
[1, 1]
-----------------------------------------------------------------------------
termination status: SLOW_PROGRESS
solution status: UNKNOWN_RESULT_STATUS
optimum = -506043.86297587276

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Found a locally optimal solution by Ipopt, giving an upper bound: 0.00005726.
The relative optimality gap is: 50604386.303313%.
Found a locally optimal solution by Ipopt, giving an upper bound: 0.00001428.
The 

(PolyVar{true}[ϵ, h_Re, h_Im, γ₁, γ₂, γ₃, a₁, a₂, a₃] => [25.14090712564675, 0.00027655267071853166, -0.00016289334268159876, 0.09202604001477685, 0.179730415114202, 0.27062819175132286, 2.0222953650550215e-5, 1.5690937922039351e-6, -0.25698177228022534], "LOCAL/FAIL")

In [19]:
Cˢⁱᵈ = convert.(ComplexF64,subs(Cˢʸᵐᵇ, solution))
Cˢⁱᵈ[1:2,1:2]

2×2 Matrix{ComplexF64}:
 0.358333+0.0im            0.0+0.256982im
      0.0-0.256982im  0.182924+0.0im

In [20]:
function min_cs_tssos(p, constrs)

    coeffs = coefficients(p)
    reg_coef = 0

    pop =[p+reg_coef*sum(variables(p).^2), constrs...] ./ maximum(abs.(coeffs))

    d = maxdegree(p)
    
    # Initial optimization step
    opt, sol, data = cs_tssos_first(pop, variables(pop), d; solution=true, QUIET=true)
    ref_sol, flag = TSSOS.refine_sol(opt, sol, data; QUIET=true)
    prev_opt, prev_sol, prev_data = opt, sol, data 

    # Check if the solution needs further refinement
    if flag != 0
        while ~isnothing(sol) && flag != 0
            prev_opt, prev_sol, prev_data = opt, sol, data
            opt, sol, data = cs_tssos_higher!(data; solution=true, QUIET=true) 
        end
        ref_sol, flag = TSSOS.refine_sol(prev_opt, prev_sol, prev_data; QUIET=true)
    end

    solution = variables(p) => ref_sol

    if flag == 0 
        status_name = "GLOBAL"
    else
        status_name = "LOCAL/FAIL"
    end

    return solution, status_name

end

min_cs_tssos (generic function with 1 method)

In [21]:
sol, st = min_cs_tssos(polyGEXYfull, constraints)

*********************************** TSSOS ***********************************
TSSOS is launching...
-----------------------------------------------------------------------------
The clique sizes of varibles:
[8, 7]
[1, 1]
-----------------------------------------------------------------------------
optimum = 4.723847436662599e-6
Global optimality certified with relative optimality gap 0.000035%!
Global optimality certified with relative optimality gap 0.000948%!


(PolyVar{true}[ϵ, h_Re, h_Im, γ₁, γ₂, γ₃, a₁, a₂, a₃] => [25.140907354747814, 0.00027691759812974284, -0.00016618249973965186, 0.09175102173290077, 0.1797277327854707, 0.2703804036979727, 2.3434179211826456e-5, 1.8369968633329967e-6, -0.2568144207276723], "GLOBAL")

In [22]:
Cˢⁱᵈ = convert.(ComplexF64,subs(Cˢʸᵐᵇ, sol))
Cˢⁱᵈ[1:2,1:2]

2×2 Matrix{ComplexF64}:
 0.358357+0.0im            0.0+0.256814im
      0.0-0.256814im  0.182404+0.0im

In [23]:
sol, st = min_cs_tssos(filter_odd_terms_by_relative_threshold(polyGEXYfull, 1e-4), constraints)

*********************************** TSSOS ***********************************
TSSOS is launching...
-----------------------------------------------------------------------------
The clique sizes of varibles:
[6, 2, 1]
[1, 2, 1]
-----------------------------------------------------------------------------
optimum = 6.090834072164834e-8
Global optimality certified with relative optimality gap 0.000011%!
Global optimality certified with relative optimality gap 0.001182%!


(PolyVar{true}[ϵ, h_Re, h_Im, γ₁, γ₂, γ₃, a₁, a₂, a₃] => [25.14068707735045, -0.02675530085210475, -0.040401917644481984, 0.17353510697067281, 0.16976514554877306, 0.25400100046242186, 0.04022190180855245, -0.026637055015705447, -0.24067737132596073], "GLOBAL")

In [24]:
st

"GLOBAL"

In [25]:
Cˢⁱᵈ = convert.(ComplexF64,subs(Cˢʸᵐᵇ, sol))
Cˢⁱᵈ[1:2,1:2]

2×2 Matrix{ComplexF64}:
 0.250231+0.0im            0.0+0.240677im
      0.0-0.240677im  0.257771+0.0im

In [26]:
Hˢⁱᵈ = convert.(ComplexF64, subs(Hˢʸᵐᵇ, sol))

2×2 Matrix{ComplexF64}:
    12.5703+0.0im       -0.0133777-0.020201im
 -0.0133777+0.020201im    -12.5703+0.0im

In [27]:
println(" SYSTEM IDENTIFICATION w CONSTRAINED TSSOS and KOSSAKOWSKI Frobenius objective QO simulation")

γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))

evol_data_file_name = "../DATA/ALL_GAMMAS_B4_D10.h5"

relative_threshold = 1e-4

rltrs = string(convert(Int, floor(log10(relative_threshold))))

tests_data_file_name = "POP_NEW_TSSOS_KOS_CONSTR_treshold_1e"*rltrs*"_FROB_QO_"*date_and_time_string * ".h5"

#relative_threshold = 0
#tests_data_file_name = "POP_NEW_TSSOS_KOS_CONSTR_NOtreshold_FROB_QO_"*date_and_time_string * ".h5"

FminGammas = []
FmedianGammas = []
FmeanGammas = []
Epsilons = []
CoefRanges = []
#Markovianity = []

for γᵢ in γ

    println("γ =  "*γᵢ)

    tᵍᵉˣʸ , ρᵍᵉˣʸ  = read_GEXY_timeevolution(evol_data_file_name, γᵢ)
    polyGEXYfull = kossak_GEXY_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)

    push!(CoefRanges, coefficient_range(polyGEXYfull))

    #polyGEXY = polyGEXYfull
    polyGEXY = filter_odd_terms_by_relative_threshold(polyGEXYfull, relative_threshold)

    @show minimum(abs.(coefficients(polyGEXY)))
    @show maximum(abs.(coefficients(polyGEXY)))

    #sol, opt_val, status = CtrlSID.cs_tssos(polyGEXY, constraints)
    sol, status = min_cs_tssos(polyGEXY, constraints)

    print(" status:", status)

    Hˢⁱᵈ = convert.(ComplexF64, subs(Hˢʸᵐᵇ, sol))
    Cˢⁱᵈ = convert.(ComplexF64,subs(Cˢʸᵐᵇ, sol))
    epsilon = subs(ϵ, sol)

    push!(Epsilons, epsilon)
    
    h5open(tests_dir*tests_data_file_name,"cw") do fid
        γ_group = create_group(fid, γᵢ) # create gamma coupling group   
        γ_group["epsilon"] = convert(Float64, epsilon)
        γ_group["H"] = convert.(ComplexF64, Hˢⁱᵈ)
        γ_group["C"] = convert.(ComplexF64, Cˢⁱᵈ)
        γ_group["status"] = status

    end

    println()

    FminStates = []
    FmedianStates = []
    FmeanStates = []

    for state in test_states # loop over initial states
        
        print(state*" ")

        start_time = time()

        tₛ, ρₛ = read_timeevolution(evol_data_file_name, state, γᵢ)
        ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)
        #bᵗˢᵗ = LiPoSID.bloch(ρₛ)
        ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
        tᵗˢᵗ = convert(Vector{Float64}, tₛ)

        ρₒ = DenseOperator(basis, ρₛ[1])
        dt = tᵗˢᵗ[2] - tᵗˢᵗ[1]
        tᵉⁿᵈ = tᵗˢᵗ[end]

        #print("effective_Lindblad_ops for Kossakowski")

        effective_Lindblad = get_lindblad_operators(convert.(ComplexF64, Cˢⁱᵈ), fᴷᴼᴺᴮ)
        effective_Lindblad_ops = [DenseOperator(basis,j) for j in effective_Lindblad]

        #print("Simulating Kossakowski")

        tout, ρ_t_kossak = timeevolution.master(convert.(Float64, tᵗˢᵗ), ρₒ, DenseOperator(basis, Hˢⁱᵈ), effective_Lindblad_ops)
        ρˢⁱᵈ  = [ρₜ.data for ρₜ in ρ_t_kossak]

        #bˢⁱᵈ = LiPoSID.bloch(ρˢⁱᵈ)

        F = LiPoSID.fidelity_series(basis, ρₛ, ρˢⁱᵈ)

        #Fᴸᴹᴱₑₓ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρˢⁱᵈ)]   

        h5open(tests_dir*tests_data_file_name,"cw") do fid
            γ_group = open_group(fid, γᵢ) # open gamma coupling group
            init_state_group = create_group(γ_group, state) # create initial state group
            init_state_group["Fidelity"] = convert.(Float64, F)
            #init_state_group["bloch_exact"] = convert.(Float64, bᵗˢᵗ)
            #init_state_group["bloch_sid"] = convert.(Float64, bˢⁱᵈ)
            init_state_group["tr_dist_grnd"] = TrDist(ρₛ[1], ρᵍ₀)
            init_state_group["time"] = tᵗˢᵗ
        end
        
        FminState = minimum(F)
        FmedianState = mean(F)
        FmeanState = mean(F)
        
        push!(FminStates, FminState)
        push!(FmedianStates, FmedianState)
        push!(FmeanStates, FmeanState)
    
    end

    # Calculate the mean
    F_mean_value = mean(FmeanStates)

    # Calculate the median
    F_median_value = median(FmedianStates)

    # Calculate the min
    F_min_value = minimum(FminStates)

    push!(FminGammas, F_min_value)
    push!(FmedianGammas, F_median_value)
    push!(FmeanGammas, F_mean_value)

    println("Median fidelity for "*γᵢ*": ", F_median_value)

end

h5open(tests_dir*tests_data_file_name,"cw") do fid
    fid["F_min"] = convert.(Float64,FminGammas)
    fid["F_median"] = convert.(Float64,FmedianGammas)
    fid["F_mean"] = convert.(Float64,FmeanGammas)
    fid["Energy"] = convert.(Float64,Epsilons)
    fid["CoefsRanges"] = convert.(Float64,CoefRanges)

end


println(tests_data_file_name)

 SYSTEM IDENTIFICATION w CONSTRAINED TSSOS and KOSSAKOWSKI Frobenius objective QO simulation
γ =  0.079477
minimum(abs.(coefficients(polyGEXY))) = 7.16715101396591e-37
maximum(abs.(coefficients(polyGEXY))) = 501.8627949039175
*********************************** TSSOS ***********************************
TSSOS is launching...
-----------------------------------------------------------------------------
The clique sizes of varibles:
[8, 7]
[1, 1]
-----------------------------------------------------------------------------
optimum = 6.890223914049384e-7
Global optimality certified with relative optimality gap 0.000025%!
Global optimality certified with relative optimality gap 0.000011%!
 status:GLOBAL
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.079477: 0.9960367690221958
γ =  0.25133
minimum(abs.(coefficients(polyGEXY))) = 4.658513032068184e-37
maximum(abs.(coefficients(polyGEXY))) = 65.37495321955478
*********************************** TSSOS ***********************************
T

 SYSTEM IDENTIFICATION w CONSTRAINED TSSOS and KOSSAKOWSKI objective QO simulation
γ =  0.079477
minimum(abs.(coefficients(polyGEXY))) = 0.00010716999563516075
maximum(abs.(coefficients(polyGEXY))) = 501.8627949039175
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.079477: 0.9976226185525447
γ =  0.25133
minimum(abs.(coefficients(polyGEXY))) = 3.45175532950157e-5
maximum(abs.(coefficients(polyGEXY))) = 65.37495321955478
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.25133: 0.996633883661821
γ =  0.79477
minimum(abs.(coefficients(polyGEXY))) = 2.1434036477811963e-6
maximum(abs.(coefficients(polyGEXY))) = 7.630493479021682
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.79477: 0.997693174205029
γ =  2.5133
minimum(abs.(coefficients(polyGEXY))) = 1.780563581016409e-7
maximum(abs.(coefficients(polyGEXY))) = 1.121995426822635
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 2.5133: 0.9990491303024693
γ =  7.9477
minim

 SYSTEM IDENTIFICATION w CONSTRAINED TSSOS and KOSSAKOWSKI objective QO simulation
γ =  0.079477
minimum(abs.(coefficients(polyGEXY))) = 7.16715101396591e-37
maximum(abs.(coefficients(polyGEXY))) = 501.8627949039175
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.079477: 0.9976226185525441
γ =  0.25133
minimum(abs.(coefficients(polyGEXY))) = 4.658513032068184e-37
maximum(abs.(coefficients(polyGEXY))) = 65.37495321955478
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.25133: 0.9966338836618186
γ =  0.79477
minimum(abs.(coefficients(polyGEXY))) = 5.527952391752069e-38
maximum(abs.(coefficients(polyGEXY))) = 7.630493479021682
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.79477: 0.9976931742050117
γ =  2.5133
minimum(abs.(coefficients(polyGEXY))) = 9.073762994668326e-39
maximum(abs.(coefficients(polyGEXY))) = 1.121995426822635
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 2.5133: 0.9990491303025519
γ =  7.9477
mi

 SYSTEM IDENTIFICATION w CONSTRAINED TSSOS and KOSSAKOWSKI objective QO simulation
γ =  0.079477
 status:Local
D1 effective_Lindblad_opsSimulating KossakowskiD2 effective_Lindblad_opsSimulating KossakowskiD3 effective_Lindblad_opsSimulating KossakowskiD4 effective_Lindblad_opsSimulating KossakowskiD5 effective_Lindblad_opsSimulating KossakowskiD6 effective_Lindblad_opsSimulating KossakowskiD7 effective_Lindblad_opsSimulating KossakowskiD8 effective_Lindblad_opsSimulating KossakowskiD9 effective_Lindblad_opsSimulating KossakowskiD10 effective_Lindblad_opsSimulating KossakowskiMedian fidelity for 0.079477: 0.9976226185525441
γ =  0.25133
 status:Local
D1 effective_Lindblad_opsSimulating KossakowskiD2 effective_Lindblad_opsSimulating KossakowskiD3 effective_Lindblad_opsSimulating KossakowskiD4 effective_Lindblad_opsSimulating KossakowskiD5 effective_Lindblad_opsSimulating KossakowskiD6 effective_Lindblad_opsSimulating KossakowskiD7 effective_Lindblad_opsSimulating KossakowskiD8 effective_

 SYSTEM IDENTIFICATION w CONSTRAINED TSSOS and KOSSAKOWSKI objective
γ =  0.079477
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.079477: 0.9977046690410768
γ =  0.25133
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.25133: 0.9966807880531685
γ =  0.79477
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.79477: 0.9977004864050036
γ =  2.5133
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 2.5133: 0.999049835240695
γ =  7.9477
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 7.9477: 0.9996748517732994
γ =  25.133
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 25.133: 0.9968645845347491
γ =  79.477
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 79.477: 0.9989415677938539
γ =  251.33
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 251.33: 0.999393310891379
POP_NEW_TSSOS_KOS_2024-Jun-15_at_04-22.h5


In [18]:
println(tests_data_file_name)

POP_NEW_TSSOS_KOS_2024-Jun-15_at_04-22.h5
